<a href="https://colab.research.google.com/github/asya99/AIHeadlines/blob/main/AIHeadlineSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# download nltk corpus (first time only)
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
##Topic modeling

import pandas as pd
import gensim #the library for Topic modelling
!pip install pyLDAvis
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim #LDA visualization library


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import pandas as pd
df = pd.read_csv('dataset.csv', header = 0, usecols=["translated_title"])
with open('pickled_data.pkl', 'wb') as f:
    # Pickle the DataFrame
    pickle.dump(df, f)

In [ ]:
df = pd.read_pickle('pickled_data.pkl')
print(df.shape)

(10000, 1)


In [ ]:
!pip install nltk
!python -m nltk.downloader stopwords
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df['preprocessTitle'] = df['translated_title'].apply(clean)

In [ ]:
print(df)

                                       translated_title  \
0     Can artificial intelligence replace small talk...   
1     People trust chatbots more easily. Emotional A...   
2     How to implement AI in online commerce? - Digi...   
3     Controlling AI: Be careful, AI! | ZEIT ONLINE ...   
4     Tips to win the Artificial Intelligence lotter...   
...                                                 ...   
9995  Charlie Munger on AI: 'I think old-fashioned i...   
9996  I went on patrol with AI police robot that wil...   
9997  EVA.ai announces new AI sidekick with 6 billio...   
9998  From AI to LLM: Establishing an orderly artifi...   
9999  Alexa Evolves with Advanced AI Technology: Ama...   

                                        preprocessTitle  
0     [artificial, intelligence, replace, small, tal...  
1     [people, trust, chatbots, easily, emotional, a...  
2        [implement, ai, online, commerce, digital, hr]  
3     [controlling, ai, careful, ai, zeit, online, z...  
4

In [ ]:
#create dictionary
dictionary = corpora.Dictionary(df['preprocessTitle'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)


91989


In [ ]:
#create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['preprocessTitle'] ]
print(len(doc_term_matrix))

10000


In [ ]:
lda = gensim.models.ldamodel.LdaModel

In [ ]:
num_topics=20
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)


CPU times: user 2min 11s, sys: 840 ms, total: 2min 12s
Wall time: 2min 26s


In [ ]:
ldamodel.print_topics(num_topics=num_topics)



[(0,
  '0.122*"ai" + 0.073*"business" + 0.025*"new" + 0.021*"insider" + 0.020*"generative" + 0.017*"post" + 0.016*"robot" + 0.013*"wire" + 0.012*"plan" + 0.011*"product"'),
 (1,
  '0.069*"ai" + 0.018*"korea" + 0.017*"samsung" + 0.017*"coming" + 0.017*"apple" + 0.014*"support" + 0.013*"guide" + 0.010*"8" + 0.010*"openais" + 0.010*"day"'),
 (2,
  '0.094*"chatbot" + 0.091*"ai" + 0.024*"musk" + 0.022*"job" + 0.021*"chatgpt" + 0.021*"say" + 0.020*"elon" + 0.019*"launch" + 0.015*"mint" + 0.014*"english"'),
 (3,
  '0.068*"risk" + 0.045*"make" + 0.040*"better" + 0.030*"washington" + 0.024*"network" + 0.022*"key" + 0.022*"american" + 0.021*"leader" + 0.019*"claude" + 0.018*"predict"'),
 (4,
  '0.261*"intelligence" + 0.260*"artificial" + 0.016*"news" + 0.008*"use" + 0.008*"new" + 0.006*"jd" + 0.006*"supra" + 0.006*"bill" + 0.006*"order" + 0.006*"executive"'),
 (5,
  '0.071*"ai" + 0.049*"open" + 0.037*"stock" + 0.035*"fool" + 0.034*"motley" + 0.033*"gpt4" + 0.020*"source" + 0.019*"challenge" + 0.

In [ ]:
# !pip install "pandas<1.2.0"
# import pyLDAvis

lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
pyLDAvis.display(lda_display)


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
lda_corpus = ldamodel[doc_term_matrix]


In [ ]:
[doc for doc in lda_corpus]


[[(0, 0.029339332),
  (1, 0.02883775),
  (2, 0.029184269),
  (3, 0.029042605),
  (4, 0.88359606)],
 [(0, 0.018584354),
  (1, 0.018362375),
  (2, 0.018529303),
  (3, 0.92616034),
  (4, 0.018363642)],
 [(0, 0.028655356),
  (1, 0.028747452),
  (2, 0.028859446),
  (3, 0.8849818),
  (4, 0.028755927)],
 [(0, 0.02048103),
  (1, 0.020665321),
  (2, 0.020694155),
  (3, 0.9177418),
  (4, 0.020417683)],
 [(0, 0.028625477),
  (1, 0.028626725),
  (2, 0.02889724),
  (3, 0.19791041),
  (4, 0.7159402)],
 [(0, 0.022374373),
  (1, 0.022377204),
  (2, 0.0223692),
  (3, 0.6841908),
  (4, 0.24868836)],
 [(0, 0.020195663),
  (1, 0.02021027),
  (2, 0.020653782),
  (3, 0.5025833),
  (4, 0.43635696)],
 [(0, 0.0202087),
  (1, 0.020301627),
  (2, 0.020286083),
  (3, 0.91902167),
  (4, 0.020181894)],
 [(0, 0.020043604),
  (1, 0.11952378),
  (2, 0.020113291),
  (3, 0.28382266),
  (4, 0.5564967)],
 [(0, 0.020054627),
  (1, 0.020302083),
  (2, 0.020769967),
  (3, 0.6304763),
  (4, 0.30839702)],
 [(0, 0.10929802),
  

In [ ]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)


0.19999999989298173


In [ ]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
print(len(cluster4))
print(len(cluster5))


NameError: name 'lda_corpus' is not defined

In [ ]:
#df.iloc[cluster1]
#!pip install pandas-flavor
##from pandas_flavor import SequenceNotStr

print(df.iloc[cluster1])
with open('cluster1.csv', 'w') as f:
    df.iloc[cluster1].to_csv(f, index=False)


                                       translated_title  \
15    Messi has 821 goals, artificial intelligence c...   
20    The Creator Film Review: Sympathy for AI - The...   
31    On the Record': Artificial Intelligence in the...   
38    How AI could save coral endangered by warming ...   
51    Why you Shouldn't Trust AI to Identify your Mu...   
...                                                 ...   
9990  Rise of artificial intelligence is inevitable ...   
9991  Are AI chatbots in courts putting justice at r...   
9995  Charlie Munger on AI: 'I think old-fashioned i...   
9996  I went on patrol with AI police robot that wil...   
9999  Alexa Evolves with Advanced AI Technology: Ama...   

                                        preprocessTitle  
15    [messi, 821, goal, artificial, intelligence, c...  
20    [creator, film, review, sympathy, ai, linfield...  
31    [record, artificial, intelligence, classroom, ...  
38    [ai, could, save, coral, endangered, warming, ...  
5